# 1.为什么要用 Batch Normalization？

在神经网络训练中，隐藏层的输入分布会随着训练过程不断变化（叫做 Internal Covariate Shift，`内部协变量偏移`），这会导致训练变慢甚至不稳定。

- `Batch Normalization` (BN) 的作用就是：

    - 将每一层的输入标准化（均值为 0，方差为 1），保证分布稳定。
    
    - 同时引入可学习的缩放因子 γ（weight）和偏移因子 β（bias），以保留模型表达能力。
    
    - 可以加速收敛，减轻梯度消失/爆炸。

# 2. nn.BatchNorm2d 的定义

In [ ]:
torch.nn.BatchNorm2d(
    num_features, 
    eps=1e-5, 
    momentum=0.1, 
    affine=True, 
    track_running_stats=True
)


| 参数                        | 作用                                                 |
| ------------------------- | -------------------------------------------------- |
| **num\_features**         | 特征图的通道数（即 `Conv2d` 的输出通道数 `out_channels`）。         |
| **eps**                   | 为了数值稳定性，在分母加上的一个小常数，防止除以 0。默认 `1e-5`。              |
| **momentum**              | 动量参数，用于更新**全局均值和方差**的移动平均。默认 `0.1`。                |
| **affine**                | 是否学习可训练的缩放参数 γ 和偏移参数 β。默认 `True`。                  |
| **track\_running\_stats** | 是否跟踪全局均值和方差（用于推理阶段）。训练时会计算 batch 内均值/方差，推理时用全局统计量。 |


### 使用示例

In [ ]:
import torch.nn as nn

# 假设输入是 (Batch, 3, 32, 32)
conv1 = nn.Conv2d(3, 64, kernel_size=3, padding=1) # 输出通道是 64
bn1 = nn.BatchNorm2d(64) # 必须指定为 64，匹配 conv1 的输出
relu1 = nn.ReLU()

# ... 后续层
conv2 = nn.Conv2d(64, 128, kernel_size=3, padding=1) # 输出通道是 128
bn2 = nn.BatchNorm2d(128) # 必须指定为 128，匹配 conv2 的输出
relu2 = nn.ReLU()

## 小细节

- 训练 vs 推理

    - 训练时：用 `batch` 内统计量。
    
    - 推理时：用整个训练过程中累计的“滑动平均统计量”。

- `num_features` 必须等于通道数，否则会报错。

- 如果 `batch size` 很小，`BN` 效果可能不好（因为均值/方差估计不准），这时可考虑 `GroupNorm `/ `LayerNorm` / `InstanceNorm`。